## preprocess

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import pickle
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import Counter
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time
import torch
torch.cuda.is_available()  # 如果返回True，说明PyTorch可以看到GPU
torch.cuda.device_count()  # 返回GPU的数量
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import os
import json
import psutil
import re
import pandas as pd
import numpy as np
import torch
from torch_geometric import utils
from torch_geometric.loader import NeighborLoader
from torch_geometric.data import Data


# # 获取当前进程，设置进程亲和性（假设你想使用CPU 17）
p = psutil.Process(os.getpid())
p.cpu_affinity([17])

%matplotlib inline

/home/cch/.conda/envs/cch_TFE_GNN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import hashlib
def stringtomd5(originstr):
    originstr = originstr.encode("utf-8")
    signaturemd5 = hashlib.sha256()
    signaturemd5.update(originstr)
    return signaturemd5.hexdigest()

In [3]:


# /home/cch/TraceSentinel/data/optc/benign/20-23Sep19/concat_SysClient0051.systemia.com.txt
with open("../data/optc/benign/20-23Sep19/benign_SysClient0201.systemia.com.txt", 'r') as file:
    content = [json.loads(line) for line in file]



def Extract_Semantic_Info(event):
    object_type = event['object']
    properties = event['properties']

    label_mapping = {
        "PROCESS": ('parent_image_path', 'image_path'),
        "FILE": ('image_path', 'file_path'),
        "FLOW": ('image_path', 'dest_ip', 'dest_port', 'direction'),
        'MODULE': ('image_path', 'module_path')
    }

    label_keys = label_mapping.get(object_type, None)
    if label_keys:
        labels = [properties.get(key) for key in label_keys]
        # print(labels)
        if all(labels):
            if object_type=="PROCESS":
                event["actorname"], event["objectname"] = labels[0], labels[1]
                event["command_line"] = properties.get("command_line", "None")
                # event["command_line"] = properties.get("command_line", "None") if len(properties.get("command_line", "None")) > 70 else properties.get("command_line", "None")
                # event["command_line"] = labels[2] if len(labels[2]) > 70 else labels[2]
            elif object_type=="FILE":
                event["actorname"], event["objectname"] = labels[0], labels[1]
            elif object_type=="FLOW":
                event["actorname"], event["objectname"] = labels[0], '-'.join(labels[1:3])
                event['action'] = labels[3]
            elif object_type == "MODULE":
                event["actorname"], event["objectname"] = labels[0], labels[1]
            else:
                pass
            return event
    return None


def Sentence_Construction(entry):
    action = entry["action"]
    properties = entry['properties']
    object_type = entry['object']
    phrase=[]
    try:
        if object_type=="PROCESS":
            phrase.append(properties.get("parent_image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(properties.get("command_line", "N/A"))
        elif object_type=="FILE":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("file_path", "N/A"))
        elif object_type=="FLOW":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            dest_ip = str(properties.get("dest_ip", "N/A"))
            dest_port = str(properties.get("dest_port", "N/A"))
            phrase.append(dest_ip+'-'+dest_port)
            # phrase.append(properties.get("src_ip", "N/A"))
            # phrase.append(properties.get("src_port", "N/A"))
            # phrase.append(properties.get("dest_ip", "N/A"))
            # phrase.append(properties.get("dest_port", "N/A"))
            # phrase.append(properties.get("direction", "N/A"))
        elif object_type=="MODULE":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("module_path", "N/A"))
        else:
            pass
    except KeyError:
        phrase = []

    return phrase

def is_valid_entry(entry):
    valid_objects = {'PROCESS', 'FILE', 'FLOW', 'MODULE'}
    invalid_actions = {'START', 'TERMINATE', 'OPEN'}

    object_valid = entry['object'] in valid_objects
    action_valid = entry['action'] not in invalid_actions
    actor_object_different = entry['actorID'] != entry['objectID']

    return object_valid and action_valid and actor_object_different

def Traversal_Rules(data):
    filtered_data = []

    for entry in data:
        if is_valid_entry(entry):
            filtered_data.append(entry)

    return filtered_data


def transform(text):
    labeled_data = [event for event in (Extract_Semantic_Info(x) for x in text) if event]
    data = Traversal_Rules(labeled_data)


    phrases = [Sentence_Construction(x) for x in data if Sentence_Construction(x)]
    for datum, phrase in zip(data, phrases):
        datum['phrase'] = phrase

    # data = sorted(data, key=lambda x: x.get('timestamp'))

    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str[:19], format='%Y-%m-%dT%H:%M:%S')
    df.sort_values(by='timestamp', inplace=True)
    df = df.reset_index(drop=True)

    return df


df = transform(content)
df.head()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,phrase,command_line
0,inbound,4f6f51c2-ff4e-4928-b73f-bb39cdf61b65,SysClient0201.systemia.com,0f0689b0-5339-4791-9751-cf971712f6d2,FLOW,0a980fb9-d7e6-4591-bd67-89b7b2b7962e,892,544,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '1', 'dest_ip': '224.0.0.251'...",-1,2019-09-19 17:11:39,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.251-5353,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
1,inbound,4f6f51c2-ff4e-4928-b73f-bb39cdf61b65,SysClient0201.systemia.com,f1c8b1f4-02d1-4c38-8b2f-15dc06fb49c8,FLOW,84a5970f-27b5-40a8-8570-70382ecbb087,892,544,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '1', 'dest_ip': '224.0.0.251'...",-1,2019-09-19 17:11:39,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.251-5353,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
2,inbound,4f6f51c2-ff4e-4928-b73f-bb39cdf61b65,SysClient0201.systemia.com,6b7c9f39-fd04-4a79-812c-1b40b43a1ae9,FLOW,20f1d8a8-690f-456b-8553-d984028e234d,892,544,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '1', 'dest_ip': '224.0.0.251'...",-1,2019-09-19 17:11:39,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.251-5353,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
3,inbound,4f6f51c2-ff4e-4928-b73f-bb39cdf61b65,SysClient0201.systemia.com,836f85f5-1cc6-40fd-bf48-71867c1524d5,FLOW,4ac9bdec-8b6a-4582-b556-28cee4851ed3,892,544,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '1', 'dest_ip': '224.0.0.251'...",-1,2019-09-19 17:11:39,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.251-5353,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN
4,inbound,4f6f51c2-ff4e-4928-b73f-bb39cdf61b65,SysClient0201.systemia.com,8c0f136f-11b1-4db6-a658-fcab280a5d7e,FLOW,af777c15-9352-4f09-a5a7-6ac7a8be924b,892,544,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '1', 'dest_ip': '224.0.0.251'...",-1,2019-09-19 17:11:39,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.251-5353,[\Device\HarddiskVolume1\Windows\system32\svch...,NaN


In [4]:
def proprocess_df(df):
    # 指定列转换为小写
    columns_to_lower = ['action', 'object', 'actorname', 'objectname', 'command_line', 'phrase']
    # df[columns_to_lower] = df[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # 使用apply和lambda函数转换列中的值
    df[columns_to_lower] = df[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # 对'phrase'列中的每个元素（列表中的字符串）应用小写转换
    df['phrase'] = df['phrase'].apply(lambda x: [item.lower() if isinstance(item, str) else item for item in x])

    node2name = {}
    # 首先遍历create事件，提取进程名
    extract_process_name = df[(df['object'] == 'process') & (df['action'] == 'create')]
    for row in extract_process_name.itertuples():
        nodeid = row.objectID
        processname = row.objectname
        if nodeid not in node2name:
            node2name[nodeid] = processname
    for row in df[(df['object'] == 'process') & (df['action'] != 'create')].itertuples():
        nodeid = row.objectID
        processname = row.objectname
        if nodeid not in node2name:
            node2name[nodeid] = processname
    # 使用 map 方法将 node2name 字典中的值应用到objectname列
    # fillna(df['objectname']) 保证如果没有对应的 nodeid，在不改变原来的值的情况下保持 objectname 原样
    df['objectname'] = df['objectID'].map(node2name).fillna(df['objectname'])
    df['actorname'] = df['actorID'].map(node2name).fillna(df['actorname'])


    new_path = {}
    for row in df[(df['object'] == 'file') & (df['action'] == 'rename')].itertuples():
        nodeid = row.objectID
        new_pathname = row.properties.get('new_path', 'None')
        if nodeid not in new_path:
            new_path[nodeid] = new_pathname

    df['objectname'] = df['objectID'].map(new_path).fillna(df['objectname'])

    # 去除重复事件
    df.drop_duplicates(subset=['action', 'actorID', 'objectID', 'object', 'actorname', 'objectname', 'pid', 'ppid'], inplace=True)
    df = df.reset_index(drop=True)

    # 根据进程名称和pid生成新的索引
    for index, row in df.iterrows():
        if row['object'] == 'process':
            # 对于 'process' 类型
            src_id = stringtomd5(row['actorname'] + str(row['ppid']))
            dest_id = stringtomd5(row['objectname'] + str(row['pid']))
        elif row['object'] in ['flow', 'file', 'module']:
            # 对于 'flow' 或 'file' 类型
            src_id = stringtomd5(row['actorname'] + str(row['pid']))
            dest_id = stringtomd5(row['objectname'])
        else:
            # 如果其他类型的 object
            src_id = None
            dest_id = None
        
        # 将结果存入新的列
        df.at[index, 'src_id'] = src_id
        df.at[index, 'dest_id'] = dest_id

    # 去除重复事件
    df.drop_duplicates(subset=['action', 'src_id', 'dest_id', 'object', 'actorname', 'objectname'], inplace=True)
    df = df[df['src_id'] != df['dest_id']]
    df = df.reset_index(drop=True)
    return df

In [5]:
df = proprocess_df(df)

In [6]:
df.head()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,phrase,command_line,src_id,dest_id
0,inbound,4f6f51c2-ff4e-4928-b73f-bb39cdf61b65,SysClient0201.systemia.com,0f0689b0-5339-4791-9751-cf971712f6d2,flow,0a980fb9-d7e6-4591-bd67-89b7b2b7962e,892,544,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '1', 'dest_ip': '224.0.0.251'...",-1,2019-09-19 17:11:39,\device\harddiskvolume1\windows\system32\svcho...,224.0.0.251-5353,[\device\harddiskvolume1\windows\system32\svch...,NaN,bbb23d203018426c1ccd42dce2cc4c954e14ca79a7dc30...,540a960fe00f02eb2e4f3087a04e3c95767d350cfc8fb5...
1,inbound,4f6f51c2-ff4e-4928-b73f-bb39cdf61b65,SysClient0201.systemia.com,2d3124d1-10a2-4b7f-9d90-42c468ee3cd4,flow,91842630-88e8-4b58-bcf0-7cb4f65126d4,892,544,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '1', 'dest_ip': 'ff02::fb', '...",-1,2019-09-19 17:11:40,\device\harddiskvolume1\windows\system32\svcho...,ff02::fb-5353,[\device\harddiskvolume1\windows\system32\svch...,NaN,bbb23d203018426c1ccd42dce2cc4c954e14ca79a7dc30...,e9279d5c69ac1643a5aff74e8c61d79ca87c277c8e44b5...
2,inbound,68891e24-4133-49cd-9551-884fa718579f,SysClient0201.systemia.com,3ef182a6-3850-452e-9da3-5acef3eb0f69,flow,70b6a611-3725-451b-aeda-3b9b10c0d6fe,2328,1832,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'dest_ip': '225.0.0.1', ...",-1,2019-09-19 17:11:40,\device\harddiskvolume1\python27\python.exe,225.0.0.1-5000,"[\device\harddiskvolume1\python27\python.exe, ...",NaN,89f52420da1b94201898e86bbec1dca3f43e767ee17713...,2e5184a3ab6979f2edb0508f3e863f7cb8e52a5a538247...
3,outbound,af5f392e-8aca-4476-b0a8-3e96d72f07c2,SysClient0201.systemia.com,1385e255-190b-447d-92ee-56f454c0ef70,flow,56860fe9-09a7-4abc-aacf-76ff80e18594,1192,544,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'dest_ip': '10.50.5.12',...",-1,2019-09-19 17:11:41,\device\harddiskvolume1\lwabeat\\lwabeat.exe,10.50.5.12-9092,"[\device\harddiskvolume1\lwabeat\\lwabeat.exe,...",NaN,557ff6224594c228cf3b61c5831038405a3509b4ba6364...,0d3542e021e03e483f815f6f5b1fccb2df07ddc18a2a77...
4,inbound,183db8ca-772a-4edb-86c6-2e76fd9514e5,SysClient0201.systemia.com,dcbb32a3-bc27-4311-842d-d3a3277c018f,flow,ca2e36d9-c576-4823-aefc-affc9b518e87,4,0,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'dest_ip': '10.50.255.25...",-1,2019-09-19 17:11:42,system,10.50.255.255-138,"[system, inbound, 10.50.255.255-138]",NaN,255baa7d3044f87213df50991bc4999b83bcb83df8de4d...,94db62338644dbea58bfa77a0e6f9c79a25d452d6e6123...


In [7]:
# df[df['object'] == 'file']['action'].value_counts()

read      221493
modify     21363
write      20808
create     14592
delete     11998
rename      2220
Name: action, dtype: int64

In [8]:
# df['object'].value_counts()

module     509201
file       292474
flow       100641
process     26133
Name: object, dtype: int64

In [7]:
df = df[df['object'] != "module"]
read_df = df[df['action'] == 'read']
write_actions = ['write', 'modify', 'create', 'rename']

# 筛选出写操作的行，并排除由 system 和 dwm.exe 修改的文件
# write_df = df[(df['action'].isin(write_actions)) & (~df['actorname'].isin(['system', 'dwm.exe']))]
sys_dwm = df[df['actorname'].isin(['system', 'dwm.exe'])]['dest_id']

write_df = df[df['action'].isin(write_actions)]



# write_object_ids = set(write_df['dest_id'])
write_object_ids = set(write_df['dest_id']) - set(sys_dwm)
readonly_df = read_df[~read_df['dest_id'].isin(write_object_ids)]
readonly_df_ids = readonly_df['dest_id']
df = df[~df['dest_id'].isin(readonly_df_ids)]

In [8]:
# 从df中去除名为\device\harddiskvolume1\windows\system32\svchost.exe和\device\harddiskvolume1\windows\system32\conhost.exe的节点
exclude_values = [
    '\device\harddiskvolume1\windows\system32\svchost.exe',
    '\device\harddiskvolume1\windows\system32\conhost.exe',
    'svchost.exe',
    'consent.exe'
]

# 过滤掉包含指定值的行
df = df[~df['actorname'].isin(exclude_values) & ~df['objectname'].isin(exclude_values)]
df = df.reset_index(drop=True)

In [9]:
# 找到所有 create 操作的父进程和子进程
parent_child_df = df[(df['action'] == 'create') & (df['object'] == 'process')]

# 提取所有父进程和子进程的 ID
parent_ids = set(parent_child_df['src_id'])  # 父进程 ID
child_ids = set(parent_child_df['dest_id'])   # 子进程 ID

# 所有相关的进程 ID（即有父进程或有子进程的）
related_ids = parent_ids.union(child_ids)

# 从 df 中移除既没有父进程也没有子进程的节点
df = df[df['src_id'].isin(related_ids) | df['dest_id'].isin(related_ids)]

# 重置索引
df = df.reset_index(drop=True)

In [10]:
# 过滤掉没有cmdline的进程节点：
df = df[~((df['object'] == 'process') & (df['command_line'] == 'none'))]

In [11]:
import pandas as pd
import networkx as nx

# 假设 df 是你的 DataFrame
# 1. 筛选 object 列为 'process' 的行
df_process = df[df['object'] == 'process']

# 2. 创建一个无向图
G = nx.Graph()

# 3. 添加边 (src_id, dest_id) 到图中
for _, row in df_process.iterrows():
    G.add_edge(row['src_id'], row['dest_id'])

# 4. 获取图中的所有连通子图
connected_components = list(nx.connected_components(G))

# 5. 过滤掉节点数小于 2 的子图
filtered_components = [comp for comp in connected_components if len(comp) > 2]

# 6. 获取所有保留的节点
valid_nodes = set(node for component in filtered_components for node in component)

# 7. 根据 valid_nodes 过滤原 DataFrame 中的行
# df_filtered = df_process[df_process['src_id'].isin(valid_nodes) & df_process['dest_id'].isin(valid_nodes)]

# 结果：df_filtered 就是筛选后的 DataFrame
df = df[(df['src_id'].isin(valid_nodes) | df['dest_id'].isin(valid_nodes))]

In [9]:
# df = df[df['object'] != "module"]

In [10]:
# df['object'].value_counts()

file       292474
flow       100641
process     26133
Name: object, dtype: int64

In [11]:
# read_df = df[df['action'] == 'read']

In [12]:
# write_actions = ['write', 'modify', 'create', 'rename']
# write_df = df[df['action'].isin(write_actions)]

In [13]:
# write_object_ids = set(write_df['dest_id'])
# readonly_df = read_df[~read_df['dest_id'].isin(write_object_ids)]

In [14]:
# readonly_df_ids = readonly_df['dest_id']
# df = df[~df['dest_id'].isin(readonly_df_ids)]

In [15]:
df['object'].value_counts()

file       184945
flow       100641
process     26133
Name: object, dtype: int64

## word2vec

In [12]:
import json
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

class EpochSaver(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('../trained_weights/optc/cmdline_attack1_w2v.model')
        self.epoch += 1

class EpochLogger(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

def prepare_sentences(df):
    nodes = {}
    for index, row in df.iterrows():
        for key in ['src_id', 'dest_id']:
            node_id = row[key]
            nodes.setdefault(node_id, []).extend(row['phrase'])
    return list(nodes.values())

def train_word2vec_model(df):
    phrases = prepare_sentences(df)

    logger = EpochLogger()
    saver = EpochSaver()
    word2vec = Word2Vec(sentences=phrases, vector_size=64, window=5, min_count=2, workers=8, epochs=50, callbacks=[saver, logger])

    return word2vec

In [13]:
train_word2vec_model(df)

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end


In [14]:
# 训练过程见optc_parser3
word2vec_weights = '../trained_weights/optc/cmdline_attack1_w2v.model'
w2vmodel = Word2Vec.load(word2vec_weights)

In [15]:
df['action'].value_counts()

inbound     76370
create      24169
outbound    13482
read        12837
modify      11172
write        9909
delete       6822
rename       1276
Name: action, dtype: int64

## featurize

In [16]:
import numpy as np


def infer(word):
    if word in  w2vmodel.wv:
        word_embeddings = w2vmodel.wv[word]
    else:
        word_embeddings = np.zeros(64)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    output_embedding = output_embedding.detach().cpu().numpy()
    return output_embedding


def Featurize(df):
    # dummies = {'process': 0, 'flow': 1, 'file': 2, 'module': 3}
    dummies = {'process': 0, 'flow': 1, 'file': 2}

    nodes = {}
    labels = {}
    lblmap = {}
    neimap = {}
    edges = []

    for index, row in df.iterrows():
        actor_id, object_id = row['src_id'], row["dest_id"]
        object_type = row['object']

        if actor_id not in nodes:
            nodes[actor_id] = row['actorname']
        if object_id not in nodes:
            nodes[object_id] = row['objectname']

        # nodes.setdefault(actor_id, []).extend(row['phrase'])
        # nodes.setdefault(object_id, []).extend(row['phrase'])

        labels[actor_id] = dummies.get('process', -1)
        labels[object_id] = dummies.get(object_type, -1)

        if actor_id not in lblmap:
            lblmap[actor_id] = row['actorname']
        if object_id not in lblmap:
            lblmap[object_id] = row['objectname']

        neimap.setdefault(actor_id, set()).add(row['objectname'])
        neimap.setdefault(object_id, set()).add(row['actorname'])

        edge_type = row['action']
        edges.append((actor_id, object_id, edge_type))

    features, feat_labels, edge_index = [], [], [[], []]
    node_index = {}

    for node, nodename in nodes.items():
        features.append(infer(nodename))
        feat_labels.append(labels[node])
        node_index[node] = len(features) - 1

    for src, dst, action in edges:
        if action=='read' or action=='inbound':
            edge_index[0].append(node_index[dst])
            edge_index[1].append(node_index[src])
        elif action=='create':
            edge_index[0].append(node_index[src])
            edge_index[1].append(node_index[dst])
            edge_index[0].append(node_index[dst])
            edge_index[1].append(node_index[src])
        else:
            edge_index[0].append(node_index[src])
            edge_index[1].append(node_index[dst])

    mapp = list(node_index.keys())

    return features, np.array(feat_labels), edge_index, mapp, lblmap, neimap

nodes,labels,edges,mapp,lbl,nemap = Featurize(df)

In [17]:
len(nodes),len(labels),len(mapp)

(49337, 49337, 49337)

In [18]:
# 定义一个字典，存储每个节点对应的特征向量
nodes_vec = {}
for i, j in zip(mapp, nodes):
    nodes_vec[i] = j

In [39]:
# 交换源节点和目标节点，因为信息汇聚是由源节点汇聚到目标节点
# edges[0], edges[1] = edges[1], edges[0]

In [19]:
df[df['object'] == 'process']['command_line'].iloc[10]

'sc  config onesyncsvc start=disabled'

In [24]:
infer('schtasks  /query /tn "kickoffautologinrandom"')

array([ 3.4708254 , -1.2484082 , -0.69898814,  0.23804194,  0.09547319,
        0.3872281 ,  0.3222387 ,  0.24613985,  0.5577434 ,  0.1049924 ,
       -2.3596153 , -0.24298401,  2.2925646 , -2.7819586 ,  4.5058675 ,
        1.3748513 , -0.32721066, -1.9327581 , -0.03271226, -2.381833  ,
        5.8508625 , -0.4474114 ,  0.841744  , -0.51245177,  2.1129937 ,
       -4.5723267 , -1.8549986 ,  1.173516  ,  1.1397079 , -2.5069451 ,
        1.2083439 , -0.13087787, -2.596347  , -1.7488282 ,  1.1060201 ,
        1.3620267 ,  1.7750661 , -6.8589807 , -3.3996022 , -1.8615474 ,
       -2.5184643 ,  1.6608161 ,  3.5593877 , -4.9322257 ,  2.0139499 ,
       -0.76103675, -0.8358026 , -3.5051167 ,  0.81226474, -0.8011651 ,
       -2.387015  ,  1.5145344 ,  0.28557062,  0.3697181 , -1.4696822 ,
       -0.3772158 , -2.8052268 , -0.8242783 ,  2.6637394 , -4.3633223 ,
       -2.763947  ,  1.2412436 , -0.89170784,  0.5798774 ], dtype=float32)

In [25]:
# # 从df中去除名为\device\harddiskvolume1\windows\system32\svchost.exe和\device\harddiskvolume1\windows\system32\conhost.exe的节点
# exclude_values = [
#     '\device\harddiskvolume1\windows\system32\svchost.exe',
#     '\device\harddiskvolume1\windows\system32\conhost.exe',
#     'svchost.exe',
#     'consent.exe'
# ]

# # 过滤掉包含指定值的行
# df = df[~df['actorname'].isin(exclude_values) & ~df['objectname'].isin(exclude_values)]
# df = df.reset_index(drop=True)

## lowest_ancestor_find

In [20]:
df_process_lineage = df[(df['action'] == 'create') & (df['object'] == 'process')]

# 记录每个进程的父进程及其时间戳和 cmdline 信息
process_parents = {}

# 进程谱系树生成
import networkx as nx

g = nx.MultiDiGraph()
node_map = {}
# count_node = 0
for index, row in df_process_lineage.iterrows():
    src, source_name, dst, dstname, etype, object_type = row['src_id'], row["actorname"],  row['dest_id'], row['objectname'], row['action'], row['object']
    command_line = row['command_line']
    if src not in node_map:
        node_map[src] = src
        g.add_node(src, type="process", pname=source_name)
    
    if dst not in node_map:
        node_map[dst] = dst
        if source_name==dstname:
            g.add_node(dst, type=object_type, pname=dstname)
        else:
            g.add_node(dst, type=object_type, pname=dstname)
    
    if g.has_edge(node_map[dst], node_map[src]):
        pass
    else:
        if g.has_edge(node_map[src], node_map[dst]):
            pass
        elif etype=='create':
            g.add_edge(node_map[src], node_map[dst], type=etype, commandline=command_line)
        else:
            pass
    if dst not in process_parents:
        process_parents[dst] = []
    process_parents[dst].append((src, command_line))

def find_lowest_common_ancestor(graph, target_pid):
    """
    查找某个进程的具有多个子进程的最低祖先。
    :param graph: NetworkX图对象
    :param target_pid: 目标进程ID
    :return: 具有多个子进程的最低祖先进程ID
    """
    # 获取目标进程的父进程
    def get_parent(pid):
        parents = process_parents.get(pid, [])
        if parents:
            return parents[0][0]  # 假设每个进程有一个父进程
        return None
    
    # 遍历并查找具有多个子进程的父进程
    def has_multiple_children(pid):
        children = list(graph.successors(pid))
        return len(children) > 1  # 如果父进程有多个子进程
    
    # 从目标进程开始，逐步向上遍历父进程
    current_pid = target_pid
    
    while True:
        parent_pid = get_parent(current_pid)
        if parent_pid is None:
            return None  # 找不到父进程，说明已经是根节点
        if has_multiple_children(parent_pid):
            # return parent_pid
            return current_pid  # 找到具有多个子进程的最低祖先
        current_pid = parent_pid  # 向上查找父进程

# 存储每个进程的最低祖先
process_lca_dict = {}



# 遍历所有进程，查找其最低祖先并存储
for node in g.nodes:
    lca = find_lowest_common_ancestor(g, node)
    if lca is not None:
        process_lca_dict[node] = process_parents.get(lca, 'meiyou')[0][1]

In [21]:
g.number_of_nodes()

9303

In [22]:
# 示例使用
process_id = 'c3c2ee2eb8bc61dce7bb2075785780f97ccdbf170f374281cceec8b4c2e5be49'  # 例如，查找进程 '1006' 的最低祖先
ancestor = find_lowest_common_ancestor(g, process_id)
if ancestor:
    print(f"Process {process_id}'s lowest ancestor with multiple children: {ancestor}")
else:
    print(f"Process {process_id} has no ancestor with multiple children.")

Process c3c2ee2eb8bc61dce7bb2075785780f97ccdbf170f374281cceec8b4c2e5be49's lowest ancestor with multiple children: c3c2ee2eb8bc61dce7bb2075785780f97ccdbf170f374281cceec8b4c2e5be49


In [23]:
print(process_lca_dict.get('c3c2ee2eb8bc61dce7bb2075785780f97ccdbf170f374281cceec8b4c2e5be49'))

c:\windows\system32\cmd.exe /c "tasklist"


In [ ]:
# 如果一个进程没有父进程，那么他的根节点是open他的进程

In [27]:
# len(process_lca_dict)

## model

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GNNWithRootEmbedding(nn.Module):
    def __init__(self, num_features, process_lca_dict, node_id_to_hash, node_hash_to_vec):
        super(GNNWithRootEmbedding, self).__init__()
        
        self.node_hash_to_vec = node_hash_to_vec # 存储节点的uuid到其初始向量的映射，其中uuid是hash值
        self.node_id_to_hash = node_id_to_hash # 存储节点的batch.n_id到节点uuid的映射，其中batch.n_id的数值，例如1，2，3，其中uuid是hash值
        self.process_lca_dict = process_lca_dict  # 存储节点与其根节点ID的映射
        self.num_features = num_features  # 节点特征维度
        
        # 第一层图卷积，输入维度为特征 + 根节点特征维度
        self.conv1 = SAGEConv(num_features, 128, normalize=True)  # 用于邻居节点特征的聚合
        # 第二层图卷积
        self.conv2 = SAGEConv(128, 64, normalize=True)
        
        # 用于根节点特征的变换矩阵W2
        self.root_transform = nn.Linear(num_features, 128)
        # 用于根节点特征的变换矩阵W3
        self.root_transform2 = nn.Linear(128, 64)

        self.linear = nn.Linear(64, 3)
    
    def root_embedding(self, node_ids):
        """
        通过根节点映射字典，直接获取根节点的特征。
        :param node_ids: 当前批次中的节点ID
        :return: 返回每个节点对应的根节点嵌入特征
        """
        # 获取节点对应的hash值
        node_hash = [self.node_id_to_hash[node_id] for node_id in node_ids]
        # 获取节点对应的根节点特征（通过process_lca_dict获得根节点ID）
        root_ids = [self.process_lca_dict.get(node_id, "meiyougengjiedian") for node_id in node_hash]
        
        # 在这里你可以调用任意方法返回根节点的特征
        # 假设根节点特征向量已经通过某种方式计算并存储，返回对应的特征
        # get_origin_embedding 函数假设返回根节点的嵌入特征
        # return torch.stack([torch.tensor(self.node_hash_to_vec.get(root_id, torch.zeros(self.num_features))) for root_id in root_ids], dim=0)
        return torch.stack([torch.tensor(infer(root_id), dtype=torch.float) for root_id in root_ids], dim=0)

    def forward(self, x, edge_index, batch_node_ids):

        x = self.encode(x, edge_index, batch_node_ids)

        x = self.linear(x)

        return F.softmax(x, dim=1)
        
    def encode(self, x, edge_index, batch_node_ids):
        """
        :param x: 节点特征矩阵
        :param edge_index: 边索引
        :param batch_node_ids: 当前批次的节点ID
        """
        # 获取当前批次节点的根节点特征
        root_embeds = self.root_embedding(batch_node_ids)

        # 第一步，聚合邻居特征
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 第二步，计算根节点特征的影响，根节点特征通过线性变换 W2
        root_embeds_transformed = self.root_transform(root_embeds.to(device))
        root_embeds_transformed = F.relu(root_embeds_transformed)
        root_embeds_transformed = F.dropout(root_embeds_transformed, p=0.2, training=self.training)

        # 最后，邻居特征加上根节点变换后的特征
        x = x + root_embeds_transformed

        # 第二层图卷积
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)


        root_embeds_transformed2 = self.root_transform2(root_embeds_transformed)
        root_embeds_transformed2 = F.relu(root_embeds_transformed2)
        root_embeds_transformed2 = F.dropout(root_embeds_transformed2, p=0.2, training=self.training)

        x = x + root_embeds_transformed2

        return x

In [25]:
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GNNWithRootEmbedding(64, process_lca_dict, mapp, nodes_vec).to(device)
# if not gnnTrain:
#     model.load_state_dict(torch.load(gnn_weights))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [26]:
from sklearn.utils import class_weight

l = np.array(labels)
class_weights = class_weight.compute_class_weight(class_weight = "balanced",classes = np.unique(l),y = l)
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
criterion = CrossEntropyLoss(weight=class_weights,reduction='mean')

graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))

In [27]:
loader = NeighborLoader(graph, num_neighbors=[-1, -1], batch_size=5000, shuffle=True)

In [28]:
def train_model(batch):
    model.train()
    optimizer.zero_grad()
    predictions = model(batch.x, batch.edge_index, batch.n_id)[:batch.batch_size]
    y = batch.y[:batch.batch_size]
    loss = criterion(predictions, y)
    loss.backward()
    optimizer.step()
    return loss.item(), batch.batch_size

def evaluate_model(batch):
    model.eval()
    with torch.no_grad():
        predictions = model(batch.x, batch.edge_index, batch.n_id)[:batch.batch_size]
        pred_labels = predictions.argmax(dim=1)
        y = batch.y[:batch.batch_size]
        correct_predictions = int((pred_labels == y).sum())
    return correct_predictions


In [29]:
gnn_weights = "../trained_weights/optc/cmdline_attack1.pth"

In [30]:
for epoch in range(20):
    total_loss = total_correct = total_nodes = 0
    
    for batch in loader:
        # print(batch.x.shape)
        loss, num_nodes = train_model(batch)
        total_loss += loss
        total_nodes += num_nodes
        total_correct += evaluate_model(batch)

    average_loss = total_loss / total_nodes
    accuracy = total_correct / total_nodes

    print(f"Epoch #{epoch}. Training Loss: {average_loss:.5f}, Accuracy: {accuracy:.5f}")
torch.save(model.state_dict(), gnn_weights)

Epoch #0. Training Loss: 0.00018, Accuracy: 0.90569
Epoch #1. Training Loss: 0.00014, Accuracy: 0.95142
Epoch #2. Training Loss: 0.00013, Accuracy: 0.95348
Epoch #3. Training Loss: 0.00013, Accuracy: 0.95604
Epoch #4. Training Loss: 0.00013, Accuracy: 0.95857
Epoch #5. Training Loss: 0.00012, Accuracy: 0.96341
Epoch #6. Training Loss: 0.00012, Accuracy: 0.96425
Epoch #7. Training Loss: 0.00012, Accuracy: 0.97144
Epoch #8. Training Loss: 0.00012, Accuracy: 0.97886
Epoch #9. Training Loss: 0.00012, Accuracy: 0.98376
Epoch #10. Training Loss: 0.00012, Accuracy: 0.98638
Epoch #11. Training Loss: 0.00012, Accuracy: 0.98922
Epoch #12. Training Loss: 0.00012, Accuracy: 0.98944
Epoch #13. Training Loss: 0.00012, Accuracy: 0.95711
Epoch #14. Training Loss: 0.00012, Accuracy: 0.96368
Epoch #15. Training Loss: 0.00012, Accuracy: 0.96826
Epoch #16. Training Loss: 0.00012, Accuracy: 0.97154
Epoch #17. Training Loss: 0.00012, Accuracy: 0.97580
Epoch #18. Training Loss: 0.00012, Accuracy: 0.98381
Epo